In [1]:
import os
import psycopg2
import torch
from torchvision import transforms
from PIL import Image
from psycopg2.extras import execute_values
import numpy as np
import torch
import torch.nn.functional as F
from PIL import Image
from pgvector.psycopg2 import register_vector
import random


In [ ]:
import psycopg2


def connect():
    conn = psycopg2.connect('REDACTED')# Create you own database and set the connection string here

    query_sql = 'SELECT VERSION()'

    cur = conn.cursor()
    return cur, conn




In [5]:
path = 'D:\\Biometric attendance\\SOCOFing\\Real'
MODEL_PATH = 'D:\\Biometric attendance\\fingerprint_model.pth'

In [6]:



# -----------------------------------------
# 1) Recreate your metric-learning model
# -----------------------------------------
import torch.nn as nn
from torchvision.models import resnet18, ResNet18_Weights

class FingerprintNet(nn.Module):
    def __init__(self, embedding_dim=128):
        super().__init__()
        self.backbone = resnet18(weights=ResNet18_Weights.DEFAULT)
        self.backbone.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
        num_ftrs = self.backbone.fc.in_features
        self.backbone.fc = nn.Linear(num_ftrs, embedding_dim)

    def forward(self, x):
        return F.normalize(self.backbone(x), p=2, dim=1)

# -----------------------------------------
# 2) Load your fine-tuned model
# -----------------------------------------
MODEL_PATH = "D:\\Biometric attendance\\fingerprint_model.pth"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = FingerprintNet(embedding_dim=128).to(device)
state = torch.load(MODEL_PATH, map_location=device, weights_only=False)
model.load_state_dict(state)
model.eval()

# -----------------------------------------
# 3) Precompute gallery embeddings
# -----------------------------------------
DATA_DIR = path
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

# gather all BMP paths
gallery_paths = []
for root, _, files in os.walk(DATA_DIR):
    for f in files:
        if f.lower().endswith(".bmp"):
            gallery_paths.append(os.path.join(root, f))
gallery_paths.sort()

# compute embeddings
gallery_records = []  # list of (file_id, [128 floats])
with torch.no_grad():
    for path in gallery_paths:
        img = Image.open(path).convert("L")
        t   = transform(img).unsqueeze(0).to(device)
        emb = model(t).cpu().numpy().flatten().tolist()
        file_id = os.path.basename(path)
        gallery_records.append((file_id, emb))



In [18]:
cur, conn = connect()

In [ ]:
cur.execute("CREATE EXTENSION IF NOT EXISTS vector;")

In [ ]:
cur.execute("""CREATE TABLE users (
    user_id INT PRIMARY KEY,
    gender VARCHAR(10) CHECK (gender IN ('M', 'F')),
    dept VARCHAR(50),
    year INT CHECK (year >= 1 AND year <= 5)
);""")


In [ ]:
conn.commit()

In [ ]:
cur.execute("""
    CREATE TABLE fingerprint_metadata (
        file_id VARCHAR(255) PRIMARY KEY,  -- e.g., 133__M_Right_index_finger.BMP
        user_id INT REFERENCES users(user_id),
        gender VARCHAR(10) CHECK (gender IN ('M', 'F')),
        hand_side VARCHAR(10) CHECK (hand_side IN ('Left', 'Right')),
        finger VARCHAR(20)  -- e.g., 'index', 'thumb', etc.
    );
""")


In [ ]:
cur.execute(
    """CREATE TABLE attendance (
    attendance_id SERIAL PRIMARY KEY,
    user_id INT REFERENCES users(user_id),
    date DATE NOT NULL DEFAULT CURRENT_DATE,
    status VARCHAR(10) NOT NULL CHECK (status IN ('present', 'absent'))
);
""")


In [19]:
cur.execute(
    """CREATE TABLE mark_attendance(
    attendance_id VARCHAR(512) PRIMARY KEY,
    user_id INT REFERENCES users(user_id),
    department VARCHAR(128) NOT NULL,
    year INT NOT NULL CHECK (year >= 1 AND year <= 5),
    subject_code VARCHAR(128) NOT NULL,
    date DATE NOT NULL DEFAULT CURRENT_DATE,
    status VARCHAR(10) NOT NULL CHECK (status IN ('present', 'absent'))
    )"""
)

In [21]:
cur.execute(
    """CREATE TABLE department(
    department VARCHAR(128),
    year INT NOT NULL CHECK (year >= 1 AND year <= 5),
    semester INT NOT NULL,
    subject_code VARCHAR(128) PRIMARY KEY,
    subject_name VARCHAR(255) NOT NULL
    )"""
)

In [36]:
# -----------------------------------------
# 4) Upsert into Postgres
# -----------------------------------------
cur, conn = connect()
register_vector(conn)  # enables pgvector support


# create table if not exists
cur.execute("""
CREATE EXTENSION IF NOT EXISTS vector;
CREATE TABLE IF NOT EXISTS public.fingerprint_embeddings_new (
  file_id TEXT PRIMARY KEY,
  embedding VECTOR(128)
);
""")

cur.execute(
    """CREATE INDEX fingerprint_embedding_new_hnsw_idx
ON fingerprint_embeddings_new
USING hnsw (embedding vector_l2_ops);
""")

# upsert all records in one batch
execute_values(cur,
    """
    INSERT INTO public.fingerprint_embeddings_new (file_id, embedding)
    VALUES %s
    ON CONFLICT (file_id) DO UPDATE
      SET embedding = EXCLUDED.embedding
    """,
    gallery_records,
    template="(%s, %s::vector)"  # tell psycopg2 that 2nd field is vector
)

conn.commit()
cur.close()
conn.close()
print(f"Upserted {len(gallery_records)} embeddings into the database")




Upserted 6000 embeddings into the database


In [39]:
conn.commit()
cur.close()
conn.close()

In [ ]:
# prompt: count the number of rows in fingerprint_embeddings table

cur.execute("SELECT COUNT(*) FROM fingerprint_embeddings;")
row_count = cur.fetchone()[0]
print(f"Number of rows in fingerprint_embeddings table: {row_count}")


Number of rows in fingerprint_embeddings table: 6000


In [ ]:
# === Mock department & year ===
departments = ['CSE', 'ECE', 'IT', 'ME','CE','CHE']
years = ['1', '2', '3', '4', '5']
user_inserted = set()

# === Load dataset ===
real_path = '/kaggle/input/socofing/SOCOFing/Real'
MODEL_PATH = '/content/fingerprint_model.pth'


# === Process each image ===
for filename in os.listdir(real_path):
    if not filename.endswith(".BMP"):
        continue

    file_id = filename  # e.g. 133__M_Right_index_finger.BMP
    user_id = int(file_id.split("__")[0])
    gender = file_id.split("__")[1].split("_")[0]
    hand = file_id.split("__")[1].split("_")[1]
    finger = file_id.split("__")[1].split("_")[2].replace(".BMP", "")

    # Extract image embedding
    img_path = os.path.join(real_path, filename)
    img = Image.open(img_path).convert("L")
    img_tensor = transform(img).unsqueeze(0).cuda()

    with torch.no_grad():
        embedding = model(img_tensor).cpu().numpy().flatten()

    embedding_str = "[" + ", ".join(map(str, embedding)) + "]"


        # === Insert user with mock data (Table 3) ===
    if user_id not in user_inserted:
        dept = random.choice(departments)
        year = random.choice(years)
        cur.execute("""
            INSERT INTO users (user_id, gender, dept, year)
            VALUES (%s, %s, %s, %s)
            ON CONFLICT (user_id) DO NOTHING;
        """, (user_id,gender, dept, year))
        user_inserted.add(user_id)

    # === Insert metadata (Table 1) ===
    cur.execute("""
        INSERT INTO fingerprint_metadata (file_id, user_id, gender, hand_side, finger)
        VALUES (%s, %s, %s, %s, %s)
        ON CONFLICT (file_id) DO NOTHING;
    """, (file_id, user_id, gender, hand, finger))







In [31]:
import pandas as pd

# Load the original CSV
df = pd.read_csv("subject_codes_with_names.csv")

# Drop the unwanted columns
df_cleaned = df.drop(columns=["Code"])

# Save the cleaned version
df_cleaned.to_csv("subject_codes_cleaned.csv", index=False)


In [28]:
conn.close()

In [32]:
cur, conn = connect()

In [ ]:
import csv

# Connect to PostgreSQL
try:
    
    # Load data from CSV and insert
    with open("subject_codes_cleaned.csv", mode='r', encoding='utf-8') as file:
        reader = csv.DictReader(file)
        inserted = 0
        for row in reader:
            cur.execute("""
                INSERT INTO department (department, year,semester, subject_code, subject_name)
                VALUES (%s, %s, %s,%s, %s)
                ON CONFLICT (subject_code) DO NOTHING;
            """, (
                row['Department'], row['Year'], row['Semester'],row['Subject_Code'], row['Subject_Name']
            ))
            inserted += 1
        print(f"{inserted} rows processed from CSV.")

    # Commit changes and close
    conn.commit()
    cur.close()
    conn.close()
    print("Data inserted successfully and connection closed.")

except Exception as e:
    print(" Error:", e)


✅ 180 rows processed from CSV.
✅ Data inserted successfully and connection closed.
